In [ ]:
# sensor efficiency
# https://www.kaggle.com/code/antonsevostianov/icecube-sensor-efficiency-feature-engineering

In [ ]:
"""SQLite-specific utility functions for use in `graphnet.data`."""

import os.path
from typing import List

import pandas as pd
import sqlalchemy
import sqlite3


def database_exists(database_path: str) -> bool:
    """Check whether database exists at `database_path`."""
    assert database_path.endswith(
        ".db"
    ), "Provided database path does not end in `.db`."
    return os.path.exists(database_path)


def database_table_exists(database_path: str, table_name: str) -> bool:
    """Check whether `table_name` exists in database at `database_path`."""
    if not database_exists(database_path):
        return False
    query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';"
    with sqlite3.connect(database_path) as conn:
        result = pd.read_sql(query, conn)
    return len(result) == 1


def run_sql_code(database_path: str, code: str) -> None:
    """Execute SQLite code.
    Args:
        database_path: Path to databases
        code: SQLite code
    """
    conn = sqlite3.connect(database_path)
    c = conn.cursor()
    c.executescript(code)
    c.close()


def save_to_sql(df: pd.DataFrame, table_name: str, database_path: str) -> None:
    """Save a dataframe `df` to a table `table_name` in SQLite `database`.
    Table must exist already.
    Args:
        df: Dataframe with data to be stored in sqlite table
        table_name: Name of table. Must exist already
        database_path: Path to SQLite database
    """
    engine = sqlalchemy.create_engine("sqlite:///" + database_path)
    df.to_sql(table_name, con=engine, index=False, if_exists="append")
    engine.dispose()


def attach_index(
    database_path: str, table_name: str, index_column: str = "event_no"
) -> None:
    """Attach the table (i.e., event) index.
    Important for query times!
    """
    code = (
        "PRAGMA foreign_keys=off;\n"
        "BEGIN TRANSACTION;\n"
        f"CREATE INDEX {index_column}_{table_name} "
        f"ON {table_name} ({index_column});\n"
        "COMMIT TRANSACTION;\n"
        "PRAGMA foreign_keys=on;"
    )
    run_sql_code(database_path, code)


def create_table(
    columns: List[str],
    table_name: str,
    database_path: str,
    *,
    index_column: str = "event_no",
    default_type: str = "NOT NULL",
    integer_primary_key: bool = True,
) -> None:
    """Create a table.
    Args:
        columns: Column names to be created in table.
        table_name: Name of the table.
        database_path: Path to the database.
        index_column: Name of the index column.
        default_type: The type used for all non-index columns.
        integer_primary_key: Whether or not to create the `index_column` with
            the `INTEGER PRIMARY KEY` type. Such a column is required to have
            unique, integer values for each row. This is appropriate when the
            table has one row per event, e.g., event-level MC truth. It is not
            appropriate for pulse map series, particle-level MC truth, and
            other such data that is expected to have more that one row per
            event (i.e., with the same index).
    """
    # Prepare column names and types
    query_columns = []
    for column in columns:
        type_ = default_type
        if column == index_column:
            if integer_primary_key:
                type_ = "INTEGER PRIMARY KEY NOT NULL"
            else:
                type_ = "NOT NULL"

        query_columns.append(f"{column} {type_}")
    query_columns_string = ", ".join(query_columns)

    # Run SQL code
    code = (
        "PRAGMA foreign_keys=off;\n"
        f"CREATE TABLE {table_name} ({query_columns_string});\n"
        "PRAGMA foreign_keys=on;"
    )
    run_sql_code(
        database_path,
        code,
    )

    # Attaching index to all non-truth-like tables (e.g., pulse maps).
    if not integer_primary_key:
        attach_index(database_path, table_name, index_column=index_column)


def create_table_and_save_to_sql(
    df: pd.DataFrame,
    table_name: str,
    database_path: str,
    *,
    index_column: str = "event_no",
    default_type: str = "NOT NULL",
    integer_primary_key: bool = True,
) -> None:
    """Create table if it doesn't exist and save dataframe to it."""
    if not database_table_exists(database_path, table_name):
        create_table(
            df.columns,
            table_name,
            database_path,
            index_column=index_column,
            default_type=default_type,
            integer_primary_key=integer_primary_key,
        )
    save_to_sql(df, table_name=table_name, database_path=database_path)

import pyarrow.parquet as pq
import sqlite3
import pandas as pd
import sqlalchemy
from tqdm import tqdm
import os
from typing import Any, Dict, List, Optional
import numpy as np

#from graphnet.data.sqlite.sqlite_utilities import create_table

def load_input(meta_batch: pd.DataFrame, input_data_folder: str) -> pd.DataFrame:
        """
        Will load the corresponding detector readings associated with the meta data batch.
        """
        batch_id = pd.unique(meta_batch['batch_id'])

        assert len(batch_id) == 1, "contains multiple batch_ids. Did you set the batch_size correctly?"
        
        detector_readings = pd.read_parquet(path = f'{input_data_folder}/batch_{batch_id[0]}.parquet')
        sensor_positions = geometry_table.loc[detector_readings['sensor_id'], ['x', 'y', 'z'] + ['absorbtion', 'scattering']]
        sensor_positions.index = detector_readings.index

        for column in sensor_positions.columns:
            if column not in detector_readings.columns:
                detector_readings[column] = sensor_positions[column]

        detector_readings['auxiliary'] = detector_readings['auxiliary'].replace({True: 1, False: 0})
        return detector_readings.reset_index()

def add_to_table(database_path: str,
                      df: pd.DataFrame,
                      table_name:  str,
                      is_primary_key: bool,
                      ) -> None:
    """Writes meta data to sqlite table. 

    Args:
        database_path (str): the path to the database file.
        df (pd.DataFrame): the dataframe that is being written to table.
        table_name (str, optional): The name of the meta table. Defaults to 'meta_table'.
        is_primary_key(bool): Must be True if each row of df corresponds to a unique event_id. Defaults to False.
    """
    try:
        create_table(   columns=  df.columns,
                        database_path = database_path, 
                        table_name = table_name,
                        integer_primary_key= is_primary_key,
                        index_column = 'event_id')
    except sqlite3.OperationalError as e:
        if 'already exists' in str(e):
            pass
        else:
            raise e
    engine = sqlalchemy.create_engine("sqlite:///" + database_path)
    df.to_sql(table_name, con=engine, index=False, if_exists="append", chunksize = 200000)
    engine.dispose()
    return

def convert_to_sqlite(meta_data_path: str,
                      database_path: str,
                      input_data_folder: str,
                      batch_size: int = 200000,
                      batch_ids: Optional[List[int]] = None,) -> None:
    """Converts a selection of the Competition's parquet files to a single sqlite database.

    Args:
        meta_data_path (str): Path to the meta data file.
        batch_size (int): the number of rows extracted from meta data file at a time. Keep low for memory efficiency.
        database_path (str): path to database. E.g. '/my_folder/data/my_new_database.db'
        input_data_folder (str): folder containing the parquet input files.
        batch_ids (List[int]): The batch_ids you want converted. Defaults to None (all batches will be converted)
    """
    if batch_ids is None:
        batch_ids = np.arange(1,661,1).to_list()
    else:
        assert isinstance(batch_ids,list), "Variable 'batch_ids' must be list."
    if not database_path.endswith('.db'):
        database_path = database_path+'.db'
    meta_data_iter = pq.ParquetFile(meta_data_path).iter_batches(batch_size = batch_size)
    batch_id = 1
    converted_batches = []
    progress_bar = tqdm(total = len(batch_ids))
    for meta_data_batch in meta_data_iter:
        if batch_id in batch_ids:
            meta_data_batch  = meta_data_batch.to_pandas()
            add_to_table(database_path = database_path,
                        df = meta_data_batch,
                        table_name='meta_table',
                        is_primary_key= True)
            pulses = load_input(meta_batch=meta_data_batch, input_data_folder= input_data_folder)
            del meta_data_batch # memory
            add_to_table(database_path = database_path,
                        df = pulses,
                        table_name='pulse_table',
                        is_primary_key= False)
            del pulses # memory
            progress_bar.update(1)
            converted_batches.append(batch_id)
        batch_id +=1
        if len(batch_ids) == len(converted_batches):
            break
    progress_bar.close()
    del meta_data_iter # memory
    print(f'Conversion Complete!. Database available at\n {database_path}')

In [ ]:
from sklearn.preprocessing import RobustScaler
from scipy.interpolate import interp1d

def ice_transparency(data_path, datum=1950):
    # Data from page 31 of https://arxiv.org/pdf/1301.5361.pdf
    # Datum is from footnote 8 of page 29
    df = pd.read_csv(data_path, delim_whitespace=True)
    df["z"] = df["depth"] - datum
    df["z_norm"] = df["z"] / 500
    df[["scattering_len_norm", "absorption_len_norm"]] = RobustScaler().fit_transform(
        df[["scattering_len", "absorption_len"]]
    )

    # These are both roughly equivalent after scaling
    f_scattering = interp1d(df["z_norm"], df["scattering_len_norm"])
    f_absorption = interp1d(df["z_norm"], df["absorption_len_norm"])
    return f_scattering, f_absorption

f_scattering, f_absorption = ice_transparency('../../../input/ice-cube2023/dataset/ice_transparency/ice_transparency.csv')
geometry_table = pd.read_csv('../../../input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')

In [ ]:
f_absorption

In [ ]:
geometry_table.head()

In [ ]:
geometry_table["absorbtion"] = f_absorption(geometry_table["z"] / 500)
geometry_table["scattering"] = f_scattering(geometry_table["z"] / 500)

In [ ]:
def detector_group(x, z):
    """
    Assigns values - deepcore, dustlayer, abovedust, underdust -
    depending on sensor coordinates x and z
    """
    
    # Define functions for each sensor category
    def is_deepcore(x, z):
        return x in {57.2, -9.68, 31.25, 72.37, 113.19, 106.94, 41.6, -10.97} or \
               (x in {46.29, 194.34, 90.49, -32.96, -77.8, 1.71, 124.97} and 
                ((z <= 186.02 and z >= 95.91) or 
                 (z <= -157 and z >= -511)))
    
    def is_dustlayer(z):
        return z <= 0 and z >= -155
    
    def is_abovedust(x, z):
        return z > 0 and not is_deepcore(x, z) and not is_dustlayer(z)
    
    def is_underdust(x, z):
        return not is_deepcore(x, z) and not is_dustlayer(z) and not is_abovedust(x, z)
    
    # Check which sensor category the coordinates belong to
    if is_deepcore(x, z):
        return "deepcore"
    
    if is_dustlayer(z):
        return "dustlayer"
    
    if is_abovedust(x, z):
        return "abovedust"
    
    return "underdust"

def relative_qe(group):
    """
    Returns a relative quantum efficiency of a sensor group according to the following rules:
    - 1.35, if sensor is deepcore;
    - 0.95, if sensor is abovedust;
    - 1.05 if sensor is underdust;
    - 0.6 if sensor is dustlayer
    """
    if group == 'deepcore':
        return 1.35
    if group == 'abovedust':
        return 0.95
    if group == 'underdust':
        return 1.05
    else:
        return 0.6


#geometry_table['sensor_group'] = geometry_table.apply(lambda row: detector_group(row['x'], row['z']), axis=1)
#geometry_table['relative_qe'] = geometry_table['sensor_group'].apply(relative_qe)

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
geometry_table['sensor_group']  = enc.fit_transform(geometry_table['sensor_group'] )

In [ ]:
input_data_folder = '../../../input/icecube-neutrinos-in-deep-ice/train'
meta_data_path = '../../../input/icecube-neutrinos-in-deep-ice/train_meta.parquet'

# database_path = f'/home/isamu/kaggledata/ICECUBE/batch_eff_1'
# convert_to_sqlite(meta_data_path,
#                   database_path=database_path,
#                   input_data_folder=input_data_folder,
#                   batch_ids = [1] )

# database_path = f'/home/isamu/kaggledata/ICECUBE/batch_eff_51'
# convert_to_sqlite(meta_data_path,
#                   database_path=database_path,
#                   input_data_folder=input_data_folder,
#                   batch_ids = [51] )

In [ ]:
%%time
bids = list(range(1,100,1))
bids.remove(51)

database_path = f'/home/isamu/kaggledata/ICECUBE/batch_eff_1_100'
convert_to_sqlite(meta_data_path,
                  database_path=database_path,
                  input_data_folder=input_data_folder,
                  batch_ids = bids)

In [ ]:
%%time
bids = list(range(100,200,1))

database_path = f'/home/isamu/kaggledata/ICECUBE/batch_eff_100_200'
convert_to_sqlite(meta_data_path,
                  database_path=database_path,
                  input_data_folder=input_data_folder,
                  batch_ids = bids)

In [ ]:
%%time
bids = list(range(200,300,1))

database_path = f'/home/isamu/kaggledata/ICECUBE/batch_eff_200_300'
convert_to_sqlite(meta_data_path,
                  database_path=database_path,
                  input_data_folder=input_data_folder,
                  batch_ids = bids)